In [ ]:
import cs1110fa18 as a
import cs2110sp19 as b
import cs4410sp20 as c
import cs4620fa20 as d
import cs4700sp20 as e
import cs4820fa20 as f

import re
import json
from sklearn.model_selection import train_test_split

data = a.data + b.data + c.data + d.data + e.data + f.data

import csv

train, test = train_test_split(data,test_size=0.15, random_state = 69) 
raw_train= r"\n\n".join(train)
raw_test= r"\n\n".join(test)
with open("train_dataset.raw", "w+") as f:
  f.write(raw_train)

with open("test_dataset.raw", "w+") as f:
  f.write(raw_test)

In [ ]:
# Install necessary libraries
!pip install pytorch_transformers
!pip install transformers
!mkdir gpt2_finetuned

     |████████████████████████████████| 184kB 9.0MB/s 
     |████████████████████████████████| 890kB 8.8MB/s 
     |████████████████████████████████| 133kB 19.5MB/s 
     |████████████████████████████████| 1.1MB 24.1MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
     |████████████████████████████████| 7.1MB 52.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c2278095e09c7f04cc22b1d343f8fcad3167cfb32da6ded25a3169c6727366b8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.34 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 1.4MB 7.5MB/s 
     |████████████████████████████████| 2.9MB 27.1MB/s 


In [ ]:
import torch
import tqdm
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
#import logging
#logging.basicConfig(level=logging.INFO)

In [ ]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the model in evaluation mode to desactivate the DropOut modules\
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [ ]:
# Generate text by adding n tokens to seed text
def generate_text(text, n):
  indexed_tokens = tokenizer.encode(text)

  tokens_tensor = torch.tensor([indexed_tokens])
  tokens_tensor = tokens_tensor.to('cuda')
  model.to('cuda')
  for i in range(n):

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    # get the predicted next sub-word (in our case, the word 'man')
    predicted_index = torch.argmax(predictions[0, -1, :]).item()
    indexed_tokens = indexed_tokens + [predicted_index]
    tokens_tensor = torch.tensor([indexed_tokens]).to("cuda")

  predicted_text = tokenizer.decode(list(indexed_tokens))
  return predicted_text


In [ ]:
""" Return the number of unique words that occur in two text samples.
Example: word_intersect_count("one two three one", "one three four") = 2
"""
def word_intersect_count(reference, candidate):
  reference_words = reference.lower().split(" ")
  candidate_words = candidate.lower().split(" ")
  return len(set(reference_words).intersection(set(candidate_words)))


In [ ]:
gpt2_intersects = []
gpt2_candidates = []
gpt2_references = []
tokens_to_generate = 50
tokens_for_seed = 12

def remove_seed(text):
  without_seed = " ".join(text.split()[tokens_for_seed:])
  return without_seed

for reference in tqdm.tqdm(test):
  # Generate text
  seed = " ".join(reference.split()[:tokens_for_seed])
  candidate = generate_text(seed, tokens_to_generate)

  # Compare intersection between two samples, excluding seed text
  intersection = word_intersect_count(remove_seed(reference), remove_seed(candidate))
  gpt2_intersects.append(intersection)
  gpt2_candidates.append(candidate)
  gpt2_references.append(reference)




100%|██████████| 828/828 [10:02<00:00,  1.37it/s]


In [ ]:
for i, c, r in zip(gpt2_intersects, gpt2_candidates, gpt2_references):
  if i > max(gpt2_intersects) - 8:
    print("\n*******WORD INTERSECT COUNT*******\n")
    print(i)
    print("\n*******GPT2*******\n")
    print(c)
    print("\n*******PIAZZA*******\n")
    print(r)
    print("\n")
import numpy as np
print(np.mean(gpt2_intersects))
print(np.median(gpt2_intersects))


*******WORD INTERSECT COUNT*******

10

*******GPT2*******

 Byte vs Word Aligned in 4/9 lecture example I understand in the past, but I wanted to show you how to use Word Aligned in 4/9 lecture example I understand in the past, but I wanted to show you how to use Word Aligned in 4/9 lecture example

I'm going to

*******PIAZZA*******

Byte vs Word Aligned in 4/9 lecture example
I understand in the example below, that to calculate sizeOfEntry, we did log264 + 7 (for the control bits) to get 13 bits. The professor mentioned that to

get this number word-aligned/byte-aligned, you add 3 more bits to get the total to 16 bits. I thought word-and byte- aligned were two different ways of

memory alignment (word aligned meant that the address has to be a multiple of 4 and byte aligned has to be a multiple of 2). So, in this case it would make sense to me to use 16 bits total for a word aligned size for the PTE but for byte-aligned, wouldn't 14 bits total be enough?



*******WORD INTERSECT CO

In [ ]:
remove_seed(data[0])


"on Anaconda's website for ChromeOS. I know that it is loosely based on Linux so I downloaded the file for that platform but I'm lost from there. Does anyone have any guidance on where to go from here? Thanks!"

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import TextDataset,DataCollatorForLanguageModeling
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
train_path = "train_dataset.raw"
test_path = "test_dataset.raw"

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)


/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:44: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")


training_args = TrainingArguments(
    output_dir="./gpt2_finetuned", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    #prediction_loss_only=True,
)

trainer.train()
trainer.save_model()

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:852: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Step,Training Loss


In [ ]:
from transformers import pipeline
import transformers
tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2')
chef = pipeline('text-generation',model='./gpt2_finetuned', tokenizer=tokenizer, config={'max_length':400})

intersects = []
candidates = []
references = []
tokens_to_generate = 50
tokens_for_seed = 12

seeds = [" ".join(reference.split()[:tokens_for_seed]) for reference in test]

candidates = chef(seeds)
candidates_copied = [candidate[0]["generated_text"] for candidate in candidates]
print(len(candidates_copied))
print(candidates_copied[:3])
1/0
'''for reference in tqdm.tqdm(test):
  # Generate text
  seed = " ".join(reference.split()[:tokens_for_seed])
  candidate = chef(seed)[0]['generated_text']'''
for candidate in candidates:

  # Compare intersection between two samples, excluding seed text
  intersection = word_intersect_count(remove_seed(reference), remove_seed(candidate))
  intersects.append(intersection)
  candidates.append(candidate)
  references.append(reference)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

828
['Q3: Indexing Newline Python I am having trouble parsing through the input of the following sentence. The first two lines in the first sentence have multiple characters in them, meaning if it is used to replace a quote with a line in a block', 'Weird behavior in interact I have an implementation for interactions that lets me change some of my behavior through my actions. For example, I have to choose which actions I will do when the object comes back or when I select something else. Or am I', 'Assert input type Do we need to enforce/ assert the input type for Do we need to enforce/ assert the input type for the next input to us?\\n\\ngram\nAre we allowed to test and debug for nested graph nodes inside']


KeyboardInterrupt: ignored

In [ ]:
for i, c, r in zip(intersects, candidates, references):
  if i > max(intersects) - 8:
    print("\n*******WORD INTERSECT COUNT*******\n")
    print(i)
    print("\n*******GPT2*******\n")
    print(c)
    print("\n*******PIAZZA*******\n")
    print(r)
    print("\n")
import numpy as np
print(np.mean(intersects))
print(np.median(intersects))

In [ ]:
len(test)

828

In [ ]:
candidates = candidates[:828]
intersects = intersects[:828]
references = references[:828]

In [ ]:
len(candidates)

828

In [ ]:
generated = []
references = []
intersects = []
for reference, candidate in zip(test, candidates):

  # Compare intersection between two samples, excluding seed text
  intersection = word_intersect_count(remove_seed(reference), remove_seed(candidate))
  intersects.append(intersection)
  generated.append(candidate)
  references.append(reference)

for i, c, r in zip(intersects, generated, references):
  if i > max(intersects) - 1:
    print("\n*******WORD INTERSECT COUNT*******\n")
    print(i)
    print("\n*******GPT2*******\n")
    print(c)
    print("\n*******PIAZZA*******\n")
    print(r)
    print("\n")
import numpy as np
print(np.mean(intersects))
print(np.median(intersects))


*******WORD INTERSECT COUNT*******

18

*******GPT2*******

q06 boarding at the same time The question says that students who are only accepted for the first time and who are not on the first day of class will be dropped off in a different class if that other person is admitted.   What may be

*******PIAZZA*******

q06 boarding at the same time
The question says that students who are going in the same direction can go at once.

To me that seems to mean that given that the light is in their favor a student should never touch any semaphore in any way because that would involve synchronizing between threads (can't even do a V operation because then the students are not entering at the same time as one V goes through first).

However if the light is not in their favor then the student would have to end up calling P on a semaphore that is at 0 in order to suspend, and that student would only be able to enter once it is no longer 0 which could only be accomplished by a student who has the l

In [ ]:
len(intersects)

828

In [ ]:
candidates[100].split()

['Normal',
 'Debugging',
 'References',
 'When',
 'comparing',
 'our',
 'normal',
 'image',
 'with',
 'the',
 'reference',
 'images,',
 'where',
 'does',
 'our',
 'normal',
 'image',
 'seem',
 'to',
 'take',
 'place?\\n\\nGetting',
 "Can't",
 'get',
 'the',
 'main',
 'page',
 'for',
 'the',
 'directory',
 'from',
 'the',
 'root?',
 'Is',
 'this',
 'really',
 'a',
 'bug',
 'in',
 'the']

In [ ]:
candidates

['Q3: Indexing Newline Python I am having trouble parsing through the input of the following sentence. The first two lines in the first sentence have multiple characters in them, meaning if it is used to replace a quote with a line in a block',
 'Weird behavior in interact I have an implementation for interactions that lets me change some of my behavior through my actions. For example, I have to choose which actions I will do when the object comes back or when I select something else. Or am I',
 'Assert input type Do we need to enforce/ assert the input type for Do we need to enforce/ assert the input type for the next input to us?\\n\\ngram\nAre we allowed to test and debug for nested graph nodes inside',
 "Web interface wrong score I passed all the tests on the autograder and it did the same for all files. I can't see why I messed up or can't fix it. Could someone please clarify what I mean?\\n\\n",
 'Assignment 2 Partners and Study Groups Hi everyone, you should all be on the same p

In [ ]:
gpt2_candidates

[' Q3: Indexing Newline Python I am having trouble parsing through the input. I am trying to parse the input and see if I can find a way to get the value of the line. I am trying to parse the input and see if I can find a way to get the value of the line.\n\nI',
 ' Weird behavior in interact I have an implementation for interactions that lets you interact with the user.\n\nThe interface is simple:\n\ninterface User { interface UserInterface { get ; set ; } }\n\nThe interface is a simple interface that allows you to interact with the user.\n\nThe interface is',
 " Assert input type Do we need to enforce/ assert the input type?\n\nThe answer is yes.\n\nThe following code snippet shows how to do this:\n\nimport { Input } from './input'; import { Input.Input } from './input/input.js'; import { Input.",
 " Web interface wrong score I passed all the tests on the autograder.\n\nI'm not sure if this is a bug or not, but I'm not sure if this is a bug or not, but I'm not sure if this is a bug or